In [4]:
!kaggle competitions download -c dspt1-predictive-modeling-challenge

 79%|█████████████████████████████▉        | 3.00M/3.81M [00:00<00:00, 6.10MB/s]
100%|██████████████████████████████████████| 3.81M/3.81M [00:00<00:00, 6.98MB/s]
  0%|                                                | 0.00/211k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 211k/211k [00:00<00:00, 7.48MB/s]
100%|████████████████████████████████████████| 948k/948k [00:00<00:00, 8.07MB/s]

  0%|                                                | 0.00/236k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 236k/236k [00:00<00:00, 11.4MB/s]


In [7]:
%%bash
if ls *.zip
then
    for z in *.zip
    do
        unzip "$z"
        rm "$z"
    done
    for c in *.csv
    do
        chmod +rwx "$c"
    done
    if ! test -d csvs
    then
        mkdir csvs
    fi
    mv *.csv csvs/
fi

test_features.csv.zip
train_features.csv.zip
train_labels.csv.zip
Archive:  test_features.csv.zip
  inflating: test_features.csv       
Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        


In [10]:
!ls csvs/

sample_submission.csv train_features.csv
test_features.csv     train_labels.csv


In [139]:
import pandas as pd
import numpy as np

In [140]:
## Read In csvs, find target.
x_train = pd.read_csv('csvs/train_features.csv')
y_train = pd.read_csv('csvs/train_labels.csv')
x_test = pd.read_csv('csvs/test_features.csv')

y_train.head(1)

,id,status_group
0,69572,functional


In [141]:
## Look at possible values for target and the distribution
target = 'status_group'
y_train[target].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [142]:
## Look for clues as to how train/test was split
x_train.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


In [143]:
x_train.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,55765,55745,59400,59400,59029,59400,59400,59400,56066,59400,55523,31234,56344,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400,59400
unique,356,1897,2145,37400,9,19287,21,125,2092,2,1,12,2696,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-15,Government Of Tanzania,DWE,none,Lake Victoria,Madukani,Iringa,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,572,9084,17402,3563,10248,508,5294,2503,307,51011,59400,36793,682,38852,26780,26780,26780,40507,52490,25348,25348,50818,50818,33186,33186,17021,17021,45794,28522,34625


In [144]:
x_test.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,14358.000000,14358.000000,14358.0000,14358.000000,1.435800e+04,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000
mean,37232.859799,324.219996,653.6363,34.082414,-5.697584e+00,0.408971,15.156359,5.713052,187.055439,1298.251985
std,21382.890432,2533.367778,688.2721,6.564449,2.947444e+00,8.231859,17.387588,9.794304,476.065978,952.551852
min,10.000000,0.000000,-57.0000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18765.500000,0.000000,0.0000,33.062317,-8.453125e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37442.000000,0.000000,346.0000,34.898976,-5.087905e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55909.250000,25.000000,1306.0000,37.221606,-3.314240e+00,0.000000,17.000000,5.000000,230.000000,2004.000000
max,74249.000000,200000.000000,2777.0000,40.325016,-2.000000e-08,669.000000,99.000000,80.000000,11469.000000,2013.000000


In [145]:
x_test.describe(exclude='number')

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,14358,13575,13570,14358,14358,14264,14358,14358,14358,13573,14358,13419,7519,13695,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358,14358
unique,331,960,1075,10615,9,8253,21,124,1934,2,1,11,1772,2,17,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,2011-03-16,Government Of Tanzania,DWE,none,Lake Victoria,Shuleni,Shinyanga,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,Borehole,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,communal standpipe,communal standpipe
freq,137,2117,4162,822,2535,136,1258,611,79,12308,14358,8807,158,9442,6168,6168,6168,9780,12639,6098,6098,12237,12237,7997,7997,4211,4211,11127,6790,8260


In [146]:
## No clear train/test split found, assuming it's random
## Begin data exploration

x_train.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [147]:
## Fills non-numeric NaNs with mode
## Fills numeric NaNs with mean
## adds is_missing column for any feature with missing values
def basic_clean(df):
    for col in df.select_dtypes(exclude='number').columns:
        if df[col].isna().sum():
            fill=df[col].mode()
            df[col]=df[col].fillna(value=str(fill))
            df[f'{col}_missing'] = df[col].isna()
    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].fillna(value=df[col].mean)
    return df
x_test = basic_clean(pd.read_csv('csvs/test_features.csv'))
x_trainval = basic_clean(pd.read_csv('csvs/train_features.csv'))
y_trainval = np.array(pd.read_csv('csvs/train_labels.csv')[target])

In [148]:
## Create Validation set

from sklearn.model_selection import train_test_split as tts
x_train, x_val, y_train, y_val = tts(x_trainval, y_trainval, test_size=.2)
assert x_train.shape[0] + x_val.shape[0] == x_trainval.shape[0]

In [163]:
## Encoding

from sklearn.pipeline import make_pipeline
import category_encoders as ce

def pre_processing(pipe, x_train, x_val, x_test):
    pipe.fit(x_train)
    
    pp_train = pd.DataFrame(pipe.transform(x_train))
    pp_train.columns = x_train.columns
    
    pp_val = pd.DataFrame(pipe.transform(x_val))
    pp_val.columns = x_val.columns
    
    pp_test = pd.DataFrame(pipe.transform(x_test))
    pp_test.columns = x_test.columns
    return (pp_train, pp_val, pp_test)

pp_train, pp_val, pp_test = pre_processing(make_pipeline(ce.OrdinalEncoder()),
                                           x_train, x_val, x_test)

In [150]:
## Modeling
from xgboost import XGBClassifier

eval_set = [(pp_train, y_train),
            (pp_val, y_val)
           ]

model = XGBClassifier(n_estimators = 2000)

model.fit(pp_train, y_train, eval_set=eval_set, eval_metric='merror', early_stopping_rounds=50)

[0]	validation_0-merror:0.301115	validation_1-merror:0.296465
Multiple eval metrics have been passed: 'validation_1-merror' will be used for early stopping.

Will train until validation_1-merror hasn't improved in 50 rounds.
[1]	validation_0-merror:0.301115	validation_1-merror:0.296465
[2]	validation_0-merror:0.300926	validation_1-merror:0.29638
[3]	validation_0-merror:0.300884	validation_1-merror:0.29638
[4]	validation_0-merror:0.300884	validation_1-merror:0.29638
[5]	validation_0-merror:0.300884	validation_1-merror:0.29638
[6]	validation_0-merror:0.299642	validation_1-merror:0.294865
[7]	validation_0-merror:0.29979	validation_1-merror:0.294781
[8]	validation_0-merror:0.2988	validation_1-merror:0.29436
[9]	validation_0-merror:0.296359	validation_1-merror:0.290657
[10]	validation_0-merror:0.295476	validation_1-merror:0.289478
[11]	validation_0-merror:0.295896	validation_1-merror:0.28931
[12]	validation_0-merror:0.295644	validation_1-merror:0.289141
[13]	validation_0-merror:0.295307	val

[128]	validation_0-merror:0.245013	validation_1-merror:0.24638
[129]	validation_0-merror:0.244739	validation_1-merror:0.246549
[130]	validation_0-merror:0.244865	validation_1-merror:0.24638
[131]	validation_0-merror:0.244676	validation_1-merror:0.246549
[132]	validation_0-merror:0.244571	validation_1-merror:0.246128
[133]	validation_0-merror:0.244318	validation_1-merror:0.245875
[134]	validation_0-merror:0.244276	validation_1-merror:0.24596
[135]	validation_0-merror:0.244255	validation_1-merror:0.246128
[136]	validation_0-merror:0.244045	validation_1-merror:0.246128
[137]	validation_0-merror:0.24396	validation_1-merror:0.246296
[138]	validation_0-merror:0.243666	validation_1-merror:0.246128
[139]	validation_0-merror:0.243561	validation_1-merror:0.24596
[140]	validation_0-merror:0.243603	validation_1-merror:0.24596
[141]	validation_0-merror:0.243371	validation_1-merror:0.244949
[142]	validation_0-merror:0.243392	validation_1-merror:0.244865
[143]	validation_0-merror:0.243329	validation_

[257]	validation_0-merror:0.23024	validation_1-merror:0.237626
[258]	validation_0-merror:0.230093	validation_1-merror:0.237626
[259]	validation_0-merror:0.230072	validation_1-merror:0.237626
[260]	validation_0-merror:0.229672	validation_1-merror:0.237205
[261]	validation_0-merror:0.22963	validation_1-merror:0.23771
[262]	validation_0-merror:0.229545	validation_1-merror:0.23771
[263]	validation_0-merror:0.229524	validation_1-merror:0.237795
[264]	validation_0-merror:0.229356	validation_1-merror:0.237626
[265]	validation_0-merror:0.229251	validation_1-merror:0.237542
[266]	validation_0-merror:0.229125	validation_1-merror:0.237626
[267]	validation_0-merror:0.229188	validation_1-merror:0.23771
[268]	validation_0-merror:0.22904	validation_1-merror:0.237458
[269]	validation_0-merror:0.228977	validation_1-merror:0.237458
[270]	validation_0-merror:0.228641	validation_1-merror:0.237374
[271]	validation_0-merror:0.228514	validation_1-merror:0.237626
[272]	validation_0-merror:0.228451	validation_

[386]	validation_0-merror:0.221002	validation_1-merror:0.235269
[387]	validation_0-merror:0.221044	validation_1-merror:0.234933
[388]	validation_0-merror:0.221002	validation_1-merror:0.235185
[389]	validation_0-merror:0.220854	validation_1-merror:0.235185
[390]	validation_0-merror:0.220896	validation_1-merror:0.235354
[391]	validation_0-merror:0.220791	validation_1-merror:0.235354
[392]	validation_0-merror:0.22077	validation_1-merror:0.235017
[393]	validation_0-merror:0.220707	validation_1-merror:0.234764
[394]	validation_0-merror:0.220707	validation_1-merror:0.234764
[395]	validation_0-merror:0.220623	validation_1-merror:0.235101
[396]	validation_0-merror:0.220497	validation_1-merror:0.235185
[397]	validation_0-merror:0.220412	validation_1-merror:0.235269
[398]	validation_0-merror:0.220244	validation_1-merror:0.235354
[399]	validation_0-merror:0.220118	validation_1-merror:0.235269
[400]	validation_0-merror:0.219844	validation_1-merror:0.235017
[401]	validation_0-merror:0.219676	valida

[515]	validation_0-merror:0.211027	validation_1-merror:0.230556
[516]	validation_0-merror:0.211027	validation_1-merror:0.230471
[517]	validation_0-merror:0.210985	validation_1-merror:0.23064
[518]	validation_0-merror:0.210985	validation_1-merror:0.230471
[519]	validation_0-merror:0.210943	validation_1-merror:0.230387
[520]	validation_0-merror:0.21088	validation_1-merror:0.230556
[521]	validation_0-merror:0.210859	validation_1-merror:0.23064
[522]	validation_0-merror:0.210774	validation_1-merror:0.230556
[523]	validation_0-merror:0.210901	validation_1-merror:0.230471
[524]	validation_0-merror:0.210711	validation_1-merror:0.230387
[525]	validation_0-merror:0.210417	validation_1-merror:0.230135
[526]	validation_0-merror:0.210501	validation_1-merror:0.230051
[527]	validation_0-merror:0.210795	validation_1-merror:0.230471
[528]	validation_0-merror:0.210795	validation_1-merror:0.230556
[529]	validation_0-merror:0.210711	validation_1-merror:0.230724
[530]	validation_0-merror:0.210732	validati

[644]	validation_0-merror:0.204945	validation_1-merror:0.228367
[645]	validation_0-merror:0.204966	validation_1-merror:0.228535
[646]	validation_0-merror:0.204903	validation_1-merror:0.228283
[647]	validation_0-merror:0.204903	validation_1-merror:0.228199
[648]	validation_0-merror:0.204819	validation_1-merror:0.228199
[649]	validation_0-merror:0.204819	validation_1-merror:0.228199
[650]	validation_0-merror:0.20484	validation_1-merror:0.228199
[651]	validation_0-merror:0.204672	validation_1-merror:0.228451
[652]	validation_0-merror:0.20463	validation_1-merror:0.228283
[653]	validation_0-merror:0.204588	validation_1-merror:0.228199
[654]	validation_0-merror:0.204609	validation_1-merror:0.228283
[655]	validation_0-merror:0.204482	validation_1-merror:0.228367
[656]	validation_0-merror:0.204398	validation_1-merror:0.228199
[657]	validation_0-merror:0.204251	validation_1-merror:0.227946
[658]	validation_0-merror:0.204125	validation_1-merror:0.227778
[659]	validation_0-merror:0.203977	validat

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=2000, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [151]:
pred_val = model.predict(pp_val)

In [152]:
## Evaluating the model
from sklearn.metrics import auc, accuracy_score

accuracy_score(pred_val,y_val)

0.7731481481481481

In [158]:
## Submission
sample = pd.read_csv('./csvs/sample_submission.csv')
sample.head(3)

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional


In [164]:
pp_test[target] = model.predict(pp_test)
submission = pp_test[['id', target]]
submission.head(3)

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,functional


In [166]:
submission.to_csv('submission_1', index=False)